## Credit default model
The purpose of this project is to create a model that is able to predict whether a loan will be defaulted based on a set of attributes of the individuals.

### Summary of results

### 0. Importing libraries and data, and general setup

In [8]:
# Importing libraries Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Keep graphs in line
%matplotlib inline
# Show all columns in pandas
pd.set_option('max_columns', None)
# Graphing style
plt.style.use('seaborn-colorblind')

In [5]:
# Importing training data provided
df_train = pd.read_csv('data/train.csv')
# Importing testing data
df_test = pd.read_csv('data/test.csv')

### 1. Exploratory data analysis

In [9]:
df_train.head()

,id,date_of_birth,number_dependants,credit_utilization,debt_to_income_ratio,monthly_income,number_open_credit_lines,number_open_loans,number_90_days_past_due,number_charged_off,score1,score2,target
0,1,1993-03-14,1,0.036495,0.208897,2400,6,2,0,0,570,817,0
1,2,1966-09-27,1,0.023423,0.260438,2200,6,1,0,0,741,756,0
2,3,1931-01-11,2,0.027205,0.335922,3000,6,1,0,1,805,779,0
3,5,1943-11-17,0,0.068424,0.314966,2500,2,3,0,0,833,629,0
4,6,1994-02-03,0,0.001003,0.251700,2500,6,6,0,1,830,587,0
